# Thesis' code

## Part 2

## 2.1 Loading the data

In [1]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import json 


In [2]:
ticker_list = ['SBER', 'GAZP', 'LKOH', 'NVTK', 'ROSN', 'GMKN', 'TATN', 'SNGS'] 

base_set = ['rv_d_past', 'rv_w_past', 'rv_m_past']
extended_set = base_set + ['high_low_past', 'overnight_past', 'rv_spx', 'rv_bz', 'r_rgbi', 'r_rtsi']

features_names = [base_set, extended_set]


In [3]:
def load_data(ticker, startdate = '20140101', enddate = '20240701'):
    # this function takes a ticker name (string) as an argument 
    # returns the table with data on that ticker
    path = 'dataset_vol/' + ticker + '.csv'
    data = pd.read_csv(path, sep = ';')
    data['date']=pd.to_datetime(data['date'])

    return data.query('date >= '+startdate+'and date < '+enddate)

data = {t: load_data(t) for t in ticker_list}

## 2.2 Modelling

In [4]:
from tqdm import tqdm

from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, root_mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, PredefinedSplit, GridSearchCV, ParameterGrid

from sklearn.preprocessing import MinMaxScaler


In [5]:
# in this cell, the functions needed to generate windows are defined
def plus_months(y, m, step):
    m = m + step
    if m > 12:
        if m % 12 == 0:
            return y + (m // 12), 12
        else:
            return y + (m // 12), m % 12
    else: 
        return y, m
    
    
def to_sd(y, m):
    if m >= 10:
        return str(y)+str(m)+'01'
    else:
        return str(y)+'0'+str(m)+'01'
        

def get_windows(data_table, features_names, to_scale=False, m_train=24, m_val=3, m_test=3, m_step=3):
    # this function given a valid dataset for one of the tickers and the list of features
    # creates a list of rolling windows, with each window split into train/validation/test
    # for the ease of future use in chosen tuning scheme, a combined dataset (train+validation) is created
    # when scaling is required, the scaler is also returned, allowing future descaling and 
    # enabling comparability of results across scaled and unscaled data
    
    windows_list = []
    
    m_window = m_train + m_val + m_test
    
    start_y = data_table.y.iloc[0]
    start_m = data_table.m.iloc[0]
    end_y = data_table.y.iloc[-1]
    end_m = data_table.m.iloc[-1]

    nw = ((12 * (end_y - start_y) + end_m - start_m + 1 - m_window) // m_step) + 1

    current_y = start_y
    current_m = start_m
    for i in range(nw):
        windows_list.append({})
        
        current_start = (current_y, current_m)
        
        for el in [('train', m_train), ('val', m_val), ('test', m_test)]:
            current_end = plus_months(current_start[0], current_start[1], el[1])
            query_str = 'date >= '+to_sd(current_start[0], current_start[1])+'and date < '+to_sd(current_end[0], current_end[1])
            q = data_table.query(query_str)
            
            windows_list[-1][el[0]+'_x'] = q[features_names].to_numpy()
            windows_list[-1][el[0]+'_y'] = q['rv_d'].to_numpy().reshape(-1, 1)
            
            current_start = current_end
        
        windows_list[-1]['comb_x'] = np.concatenate([windows_list[-1]['train_x'], windows_list[-1]['val_x']], axis=0)
        windows_list[-1]['comb_y'] = np.concatenate([windows_list[-1]['train_y'], windows_list[-1]['val_y']], axis=0)
                                          
        windows_list[-1]['test_y_scaler'] = None
        current_y, current_m = plus_months(current_y, current_m, m_step)
    
        if to_scale:
            s_train_x = MinMaxScaler()
            s_train_y = MinMaxScaler()
        
            windows_list[-1]['train_x'] = s_train_x.fit_transform(windows_list[-1]['train_x'])
            windows_list[-1]['train_y'] = s_train_y.fit_transform(windows_list[-1]['train_y'])
        
            windows_list[-1]['val_x'] = s_train_x.transform(windows_list[-1]['val_x'])
            windows_list[-1]['val_y'] = s_train_y.transform(windows_list[-1]['val_y'])
        
            s_comb_x = MinMaxScaler()
            s_comb_y = MinMaxScaler()
        
            windows_list[-1]['comb_x'] = s_comb_x.fit_transform(windows_list[-1]['comb_x'])
            windows_list[-1]['comb_y'] = s_comb_y.fit_transform(windows_list[-1]['comb_y'])
        
            windows_list[-1]['test_x'] = s_comb_x.transform(windows_list[-1]['test_x'])
        
            windows_list[-1]['test_y_scaler'] = s_comb_y
        
    
    return windows_list


In [6]:
# these two simple functions are defined to simplify the future functions
def add_metrics(values, preds, errors, metrics):
    if 'mse' in metrics:
        errors['mse'].append(float(mean_squared_error(values, preds)))
        
    if 'rmse' in metrics:
        errors['rmse'].append(float(root_mean_squared_error(values, preds)))

    if 'mae' in metrics:
        errors['mae'].append(float(mean_absolute_error(values, preds)))
            
    if 'mape' in metrics:
        errors['mape'].append(float(mean_absolute_percentage_error(values, preds)))

    if 'r2' in metrics:
        errors['r2'].append(float(r2_score(values, preds)))
    
    return errors


def calculate_metrics(values, preds, metrics):
    result = {}
    if 'mse' in metrics:
        result['mse'] = float(mean_squared_error(values, preds))
        
    if 'rmse' in metrics:
        result['rmse'] = float(root_mean_squared_error(values, preds))

    if 'mae' in metrics:
        result['mae'] = float(mean_absolute_error(values, preds))
            
    if 'mape' in metrics:
        result['mape'] = float(mean_absolute_percentage_error(values, preds))

    if 'r2' in metrics:
        result['r2'] = float(r2_score(values, preds))
    
    return result

### HAR

In [11]:
from sklearn.linear_model import LinearRegression

In [12]:
def run_har(windows, metrics = ['mse', 'rmse', 'mae', 'mape', 'r2']):
    # this function, as others from these category (run_model_name), 
    # takes as an input the windows, designed to be generated by the previously defined function
    # and outputs a tuple constituting a thorough description of the forecast 
    coefs = []
    window_metrics = {m:[] for m in metrics}
    preds = []
    targets = []
    for w in windows:
        model = LinearRegression(fit_intercept=True)
        model.fit(w['comb_x'], w['comb_y'])
        
        coefs.append(model.coef_)
        
        pred = model.predict(w['test_x'])
        
        if w['test_y_scaler']:
            pred = w['test_y_scaler'].inverse_transform(pred)
        
        preds.append(pred)
        targets.append(w['test_y'])
        
        window_metrics = add_metrics(w['test_y'], pred, window_metrics, metrics)

        
    coefs = np.array(coefs)
    preds = np.concatenate(preds)     
    targets = np.concatenate(targets) 
    result = calculate_metrics(targets, preds, metrics) 
    
    return result, window_metrics, coefs, preds, targets



In [29]:
# the piece of code in this cell is used to run the model using the custom function defined above
# and save the results (loading and processing the results is given in notebook 3)

for t in tqdm(ticker_list):                  
    for features, set_name in zip(features_names, ['rv', 'all']):
        result, window_metrics, coefs, preds, targets = run_har(get_windows(data[t], features_names=features, to_scale = False), metrics = ['mse', 'rmse', 'mae', 'mape', 'r2'])
    
        description = {'model': 'HAR',
                       'ticker': t,
                       'data_set': set_name,
                       'rolling_window': '24m3m3m',
                       'result': result,
                       'window_metrics': window_metrics,
                       'coefficients': coefs.tolist(),
                       'predictions': preds.tolist()}
    
        res_path = 'results/HAR/' + t + '_' + set_name + '.json'
    
        with open(res_path, 'w') as f:
            json.dump(description, f)
        
print('\nfinished')



100%|█████████████████████████████████████████████| 8/8 [00:06<00:00,  1.30it/s]


finished


### Lasso

In [13]:
from sklearn.linear_model import Lasso


In [14]:
def run_lasso(windows, metrics = ['mse', 'rmse', 'mae', 'mape', 'r2']):
    window_metrics = {m:[] for m in metrics}
    coefs = []
    params = []
    parameters = {'alpha': np.logspace(-5, 0, 100)}
    preds = []
    targets = []
    for w in windows:
        s_i = [-1]*w['train_x'].shape[0] + [0]*w['val_x'].shape[0]
        spl = PredefinedSplit(test_fold = s_i)
        
        model = Lasso(fit_intercept=True, random_state=0)
        gs = GridSearchCV(estimator=model, cv=spl, param_grid=parameters, refit=True, scoring='neg_mean_squared_error', n_jobs=-1)
        gs.fit(w['comb_x'], w['comb_y'])
        
        params.append(gs.best_params_)
        coefs.append(gs.best_estimator_.coef_)

        pred = gs.predict(w['test_x'])
        if w['test_y_scaler']:
            pred = w['test_y_scaler'].inverse_transform(pred.reshape(-1, 1))
        preds.append(pred)
        targets.append(w['test_y'])
        
        window_metrics = add_metrics(w['test_y'], pred, window_metrics, metrics)
            
    coefs = np.array(coefs)
    preds = np.concatenate(preds)     
    targets = np.concatenate(targets) 
    result = calculate_metrics(targets, preds, metrics) 
    
    return result, window_metrics, coefs, params, preds, targets



In [46]:
# this code, used to obtain and save the Lasso results, 
# was run both with to_scale=True and to_scale=False, and a corresponding change in the description and result path
# same applies to analogic cells below

for t in tqdm(ticker_list):
    for features, set_name in zip(features_names, ['rv', 'all']):
        result, window_metrics, coefs, params, preds, targets = run_lasso(get_windows(data[t], features_names=features, to_scale = False), metrics = ['mse', 'rmse', 'mae', 'mape', 'r2'])
    
        description = {'model': 'Lasso',
                       'ticker': t,
                       'data_scaled': False,
                       'data_set': set_name,
                       'rolling_window': '24m3m3m',
                       'result': result,
                       'window_metrics': window_metrics,
                       'coefficients': coefs.tolist(),
                       'parameters': params,
                       'predictions': preds.tolist()}
    
        res_path = 'results/Lasso/' + t + '_' + set_name + '.json'
    
        with open(res_path, 'w') as f:
            json.dump(description, f)
    
    
print('\nfinished')

100%|█████████████████████████████████████████████| 8/8 [00:43<00:00,  5.43s/it]


finished


### Random Forest

In [15]:
from sklearn.ensemble import RandomForestRegressor


In [16]:
def run_rf(windows, features_names=features_names, metrics = ['mse', 'rmse', 'mae', 'mape', 'r2']):
    window_metrics = {m:[] for m in metrics}    
    params = []
    preds = []
    targets = []

    parameters = {'n_estimators': [100, 200, 300],
                  'max_depth': [3, 5]}

    for w in tqdm(windows):
        s_i = [-1]*w['train_x'].shape[0] + [0]*w['val_x'].shape[0]
        spl = PredefinedSplit(test_fold = s_i)
                
        model = RandomForestRegressor(random_state=0)

        gs = GridSearchCV(estimator=model, cv=spl, param_grid=parameters, refit=True, scoring='neg_mean_squared_error', n_jobs=-1)
        gs.fit(w['comb_x'], w['comb_y'].reshape(-1,))
        
        params.append(gs.best_params_)
        
        pred = gs.predict(w['test_x'])
        if w['test_y_scaler']:
            pred = w['test_y_scaler'].inverse_transform(pred.reshape(-1, 1))

        preds.append(pred)
        targets.append(w['test_y'])
        
        window_metrics = add_metrics(w['test_y'], pred, window_metrics, metrics)
         
    preds = np.concatenate(preds)     
    targets = np.concatenate(targets) 
    result = calculate_metrics(targets, preds, metrics) 
    
    return result, window_metrics, params, preds, targets


In [48]:
for t in ticker_list:      
    for features, set_name in zip(features_names, ['rv', 'all']):
        result, window_metrics, params, preds, targets = run_rf(get_windows(data[t], features_names=features, to_scale = True), metrics = ['mse', 'rmse', 'mae', 'mape', 'r2'])

        description = {'model': 'RF',
                       'ticker': t,
                       'data_scaled': True,
                       'data_set': set_name,
                       'rolling_window': '24m3m3m',
                       'result': result,
                       'window_metrics': window_metrics,
                       'parameters': params,
                       'predictions': preds.tolist()}
    
        res_path = 'results/RF_s/' + t + '_' + set_name + '.json'
    
        with open(res_path, 'w') as f:
            json.dump(description, f)
    
    print(t, end=' ')
    
print('\nfinished')

100%|███████████████████████████████████████████| 33/33 [00:39<00:00,  1.18s/it]


SBER 

100%|███████████████████████████████████████████| 33/33 [00:35<00:00,  1.07s/it]


GAZP 

100%|███████████████████████████████████████████| 33/33 [00:36<00:00,  1.12s/it]


...

GMKN 

100%|███████████████████████████████████████████| 33/33 [00:35<00:00,  1.08s/it]


TATN 

100%|███████████████████████████████████████████| 33/33 [00:33<00:00,  1.01s/it]

SNGS 
finished


### XGBoost

In [17]:
import xgboost as xgb


In [18]:
def run_xgb(windows, features_names=features_names, metrics = ['mse', 'rmse', 'mae', 'mape', 'r2']):
    window_metrics = {m:[] for m in metrics}    
    params = []
    preds = []
    targets = []

    parameters = {'n_estimators': [100, 200, 300],
                  'max_depth': [3, 5],
                  'learning_rate': np.logspace(-4, -1, 20)}

    for w in tqdm(windows):
        s_i = [-1]*w['train_x'].shape[0] + [0]*w['val_x'].shape[0]
        spl = PredefinedSplit(test_fold = s_i)
        
        model = xgb.XGBRegressor(random_state=0)                
                              
        gs = GridSearchCV(estimator=model, cv=spl, param_grid=parameters, refit=True, scoring='neg_mean_squared_error', n_jobs=-1)
        gs.fit(w['comb_x'], w['comb_y'])
        
        params.append(gs.best_params_)
        
        pred = gs.predict(w['test_x'])
        if w['test_y_scaler']:
            pred = w['test_y_scaler'].inverse_transform(pred.reshape(-1, 1))
        preds.append(pred)
        targets.append(w['test_y'])
        
        window_metrics = add_metrics(w['test_y'], pred, window_metrics, metrics)

    preds = np.concatenate(preds)     
    targets = np.concatenate(targets) 
    result = calculate_metrics(targets, preds, metrics) 
    
    return result, window_metrics, params, preds, targets


In [54]:
for t in ticker_list:      
    for features, set_name in zip(features_names, ['rv', 'all']):
        result, window_metrics, params, preds, targets = run_xgb(get_windows(data[t], features_names=features, to_scale=True), metrics = ['mse', 'rmse', 'mae', 'mape', 'r2'])
        description = {'model': 'XGB',
                       'ticker': t,
                       'data_scaled': True,
                       'data_set': set_name,
                       'rolling_window': '24m3m3m',
                       'result': result,
                       'window_metrics': window_metrics,
                       'parameters': params,
                       'predictions': preds.tolist()}
    
        res_path = 'results/XGB_s/' + t + '_' + set_name + '.json'
    
        with open(res_path, 'w') as f:
            json.dump(description, f)
    
    print(t, end=' ')
    
print('\nfinished')

100%|███████████████████████████████████████████| 33/33 [01:37<00:00,  2.96s/it]


SBER 

100%|███████████████████████████████████████████| 33/33 [01:36<00:00,  2.93s/it]


GAZP 

100%|███████████████████████████████████████████| 33/33 [01:35<00:00,  2.88s/it]


...

GMKN 

100%|███████████████████████████████████████████| 33/33 [01:38<00:00,  2.98s/it]


TATN 

100%|███████████████████████████████████████████| 33/33 [01:39<00:00,  3.01s/it]

SNGS 
finished


### NN


In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader, TensorDataset

In [20]:
class neural_net(nn.Module):
    def __init__(self, input_size, n_hidden):
        super(neural_net, self).__init__()
        self.fc1 = nn.Linear(input_size, n_hidden)
        self.fc_out = nn.Linear(n_hidden, 1) 

    def forward(self, x):
        x = F.relu(self.fc1(x)) 
        x = self.fc_out(x)
        return x 
    
    
def train(model, device, train_loader, loss_fn, optimizer):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs) 
        loss = loss_fn(outputs, targets) 
        loss.backward() 
        optimizer.step()
        running_loss += loss.item() 
        
                    
def test(model, device, test_loader, loss_fn, predict = False, y_scaler=None):
    model.eval()
    test_loss = 0.0
    pred = []
    targ = []
    with torch.no_grad(): 
        for inputs, targets in test_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs) 
            if predict and y_scaler:
                outputs = torch.tensor(y_scaler.inverse_transform(outputs)).to(device)
            loss = loss_fn(outputs, targets) 
            test_loss += loss.item() 
            pred.append(outputs.detach().cpu().numpy())
            targ.append(targets.detach().cpu().numpy())
    if predict:
        return test_loss/len(test_loader), np.concatenate(pred, axis = 0), np.concatenate(targ, axis = 0)

    else:
        return test_loss/len(test_loader)
        

In [21]:
def run_nn(windows, n_hidden, metrics = ['mse', 'rmse', 'mae', 'mape', 'r2']):
    window_metrics = {m:[] for m in metrics}    
    params = []
    preds = []
    targets = []
    #errors = []
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    np.random.seed(0)
    torch.manual_seed(0)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(0)
        
    n_f = windows[0]['train_x'].shape[1]
    batch_size = 32
    for w in tqdm(windows):

        x_tr = w['train_x'] 
        y_tr = w['train_y']  
        
        x_v = w['val_x'] 
        y_v = w['val_y'] 
        
        x_c = w['comb_x'] 
        y_c = w['comb_y']  
        
        x_te = w['test_x'] 
        y_te = w['test_y']  

        train_dataset = TensorDataset(torch.tensor(x_tr, dtype=torch.float32), 
                                      torch.tensor(y_tr, dtype=torch.float32))
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
        
        val_dataset = TensorDataset(torch.tensor(x_v, dtype=torch.float32), 
                                    torch.tensor(y_v, dtype=torch.float32))
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
                
        comb_dataset = TensorDataset(torch.tensor(x_c, dtype=torch.float32), 
                                    torch.tensor(y_c, dtype=torch.float32))
        comb_loader = DataLoader(comb_dataset, batch_size=batch_size, shuffle=False)
        
        test_dataset = TensorDataset(torch.tensor(x_te, dtype=torch.float32), 
                                     torch.tensor(y_te, dtype=torch.float32))
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)        

        best_pars = []
        best_e = np.inf
        for lr in [1e-3]:
            model = neural_net(n_f, n_hidden).to(device)
            loss_fn = nn.MSELoss(reduction='mean')
            optimizer = optim.Adam(model.parameters(), lr=lr)  
            
            for epoch in range(200):
                train(model, device, train_loader, loss_fn, optimizer) 
                if (epoch + 1) % 25 == 0:
                    e_v = test(model, device, val_loader, loss_fn) 
                    if best_e > e_v:
                        best_e = e_v
                        best_pars = [epoch, lr]

        params.append(best_pars)
        
        model = neural_net(n_f, n_hidden).to(device)
        loss_fn = nn.MSELoss(reduction='mean')
        optimizer = optim.Adam(model.parameters(), lr=best_pars[1])  
        
        for epoch in range(best_pars[0]+1):
            train(model, device, comb_loader, loss_fn, optimizer) 
            
        if w['test_y_scaler']:
            err, pred, targ = test(model, device, test_loader, loss_fn, predict=True, y_scaler = w['test_y_scaler'])
        else: 
            err, pred, targ = test(model, device, test_loader, loss_fn, predict=True)
        #errors.append(err)
        preds.append(pred)
        targets.append(targ)
        
        window_metrics = add_metrics(targ, pred, window_metrics, metrics)

    preds = np.concatenate(preds)     
    targets = np.concatenate(targets) 
    result = calculate_metrics(targets, preds, metrics)   
    
    return result, window_metrics, params, preds, targets #errors



In [20]:
# here, to_scale and n_hidden could be different, with the corresponding changes to description and path

for t in ticker_list:      
    for features, set_name in zip(features_names, ['rv', 'all']):
        result, window_metrics, params, preds, targets = run_nn(get_windows(data[t], features_names=features, to_scale = True), 
                                                                n_hidden=16, metrics = ['mse', 'rmse', 'mae', 'mape', 'r2'])
        description = {'model': 'NN',
                       'ticker': t,
                       'data_scaled': True,
                       'data_set': set_name,
                       'rolling_window': '24m3m3m',
                       'n_hidden': 16,
                       'max_epoch': 200,
                       'result': result,
                       'window_metrics': window_metrics,
                       'parameters': params,
                       'predictions': preds.tolist()}
    
        res_path = 'results/NN_s_16/' + t + '_' + set_name + '.json'
    
        with open(res_path, 'w') as f:
            json.dump(description, f)
    
    print(t)
    
print('finished')

100%|███████████████████████████████████████████| 33/33 [01:55<00:00,  3.50s/it]


SBER


100%|███████████████████████████████████████████| 33/33 [02:00<00:00,  3.65s/it]


GAZP


100%|███████████████████████████████████████████| 33/33 [02:00<00:00,  3.64s/it]


...

GMKN


100%|███████████████████████████████████████████| 33/33 [01:53<00:00,  3.45s/it]


TATN


100%|███████████████████████████████████████████| 33/33 [01:56<00:00,  3.54s/it]

SNGS
finished


### RNNs


In [59]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset
from sklearn.metrics import mean_squared_error, r2_score

In [60]:
# since rnns require different input format, with the data set split into (overlapping) sequences with fixed legnths,
# a separate function for creating windows from the loaded data was defined
# it requires same arguments as the ordinary (non rnn) version plus a length of sequence (len_seq) parameter

def get_windows_rnn(data_table, features_names, to_scale=False, len_seq=5, m_train = 24, m_val = 3, m_test = 3, m_step = 3):
    x = []
    y = []
    
    for i in range(len(data_table)-len_seq+1):
        end = i + len_seq
        seq_x, seq_y = data_table.iloc[i:end][features_names], data_table.iloc[end-1]['rv_d']
        x.append(seq_x)
        y.append(seq_y)    
    
    windows_list = []
    
    len_f = len(features_names)
    m_window = m_train + m_val + m_test
    
    start_y = data_table.y.iloc[0]
    start_m = data_table.m.iloc[0]
    end_y = data_table.y.iloc[-1]
    end_m = data_table.m.iloc[-1]

    nw = ((12 * (end_y - start_y) + end_m - start_m + 1 - m_window) // m_step) + 1

    current_y = start_y
    current_m = start_m
    
    sequence_index = 0
    for i in range(nw):
        
        if sequence_index >= len(x):
            break
            
        windows_list.append({})
        
        current_start = (current_y, current_m)
        
        for el in [('train', m_train), ('val', m_val), ('test', m_test)]:
            
            current_end = plus_months(current_start[0], current_start[1], el[1])
            query_str = 'date >= '+to_sd(current_start[0], current_start[1])+'and date < '+to_sd(current_end[0], current_end[1])

            query_indices = data_table.query(query_str).index
            s_start = max(query_indices[0] - len_seq + 1, 0)
            s_end = query_indices[-1] - len_seq + 2
            
            current_start = current_end
            windows_list[-1][el[0]+'_x'] = np.array(x[s_start:s_end])
            windows_list[-1][el[0]+'_y'] = np.array(y[s_start:s_end]).reshape(-1, 1)
            
            
        current_y, current_m = plus_months(current_y, current_m, m_step)
        
        windows_list[-1]['comb_x'] = np.concatenate([windows_list[-1]['train_x'], windows_list[-1]['val_x']], axis=0)
        windows_list[-1]['comb_y'] = np.concatenate([windows_list[-1]['train_y'], windows_list[-1]['val_y']], axis=0)
                                          
        windows_list[-1]['test_y_scaler'] = None

        if to_scale:
            
            s_train_x = MinMaxScaler()
            s_train_y = MinMaxScaler()
        
            windows_list[-1]['train_x'] = s_train_x.fit_transform(np.vstack(windows_list[-1]['train_x'])).reshape(-1, len_seq, len_f)
            windows_list[-1]['train_y'] = s_train_y.fit_transform(windows_list[-1]['train_y'].reshape(-1, 1))
        
            windows_list[-1]['val_x'] = s_train_x.transform(np.vstack(windows_list[-1]['val_x'])).reshape(-1, len_seq, len_f)
            windows_list[-1]['val_y'] = s_train_y.transform(np.vstack(windows_list[-1]['val_y']))
        
            s_comb_x = MinMaxScaler()
            s_comb_y = MinMaxScaler()
        
            windows_list[-1]['comb_x'] = s_comb_x.fit_transform(np.vstack(windows_list[-1]['comb_x'])).reshape(-1, len_seq, len_f)
            windows_list[-1]['comb_y'] = s_comb_y.fit_transform(np.vstack(windows_list[-1]['comb_y']))
        
            windows_list[-1]['test_x'] = s_comb_x.transform(np.vstack(windows_list[-1]['test_x'])).reshape(-1, len_seq, len_f)
            windows_list[-1]['test_y'] = np.vstack(windows_list[-1]['test_y'])
                
            windows_list[-1]['test_y_scaler'] = s_comb_y
        
    return windows_list



In [61]:
# since the train and test functions for all the RNNs are the same, they are defined here

def train(model, device, train_loader, loss_fn, optimizer):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs) 
        loss = loss_fn(outputs, targets) 
        loss.backward() 
        optimizer.step() 
        running_loss += loss.item() 

def test(model, device, test_loader, loss_fn, predict = False, y_scaler=None):
    model.eval()
    test_loss = 0.0
    pred = []
    targ = []
    with torch.no_grad(): 
        for inputs, targets in test_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs) 
            if predict and y_scaler:
                outputs = torch.tensor(y_scaler.inverse_transform(outputs)).to(device)
            loss = loss_fn(outputs, targets) 
            test_loss += loss.item() 
            pred.append(outputs.detach().cpu().numpy())
            targ.append(targets.detach().cpu().numpy())
    if predict:
        return test_loss/len(test_loader), np.concatenate(pred, axis = 0), np.concatenate(targ, axis = 0)

    else:
        return test_loss/len(test_loader)

### Jordan RNN

In [62]:
class jordan_rnn(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(jordan_rnn, self).__init__()
        self.hidden_size = hidden_size
        self.rec = nn.Linear(input_size + 1, hidden_size)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        batch_size, seq_len, input_size = x.size()
        device = x.device

        outputs = []
        feedback = torch.zeros(batch_size, 1).to(device)

        for t in range(seq_len):
            combined_input = torch.cat((x[:, t, :], feedback), dim=1)  
            hidden = torch.relu(self.rec(combined_input))
            output = self.fc(hidden)
            feedback = output.detach()
            outputs.append(output)

        outputs = torch.stack(outputs, dim=1) 
        return outputs[:, -1, :]

### Elman NN

In [63]:
class elman_rnn(nn.Module):
    def __init__(self, input_size, hidden_size=20, num_layers=1, output_size=1):
        super(elman_rnn, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, nonlinearity='tanh')
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out

### LSTM

In [64]:
class lstm_rnn(nn.Module):
    def __init__(self, input_size, hidden_size=50):
        super(lstm_rnn, self).__init__()
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size, hidden_size, 1, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size)
        c0 = torch.zeros(1, x.size(0), self.hidden_size)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out


### GRU

In [65]:
class gru_rnn(nn.Module):
    def __init__(self, input_size, hidden_size=20):
        super(gru_rnn, self).__init__()
        self.hidden_size = hidden_size
        
        self.gru = nn.GRU(input_size, hidden_size, 1, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)

        
        out, _ = self.gru(x, h0)
        out = self.fc(out[:, -1, :])
        return out

### Running RNNs and saving results

In [66]:
def run_rnn(rnn, windows, hidden_size=20, metrics = ['mse', 'rmse', 'mae', 'mape', 'r2']):
    # this function's first argument is a string with model name, one of 'jnn', 'enn', 'lstm', 'gru'
    # other arguments are windows (should be generated by the rnn-specific function) and hidden size
    # outputs a tuple describing the forecasts
    
    window_metrics = {m:[] for m in metrics}    
    params = []
    preds = []
    targets = []
    #errors = []
    
    input_size = windows[0]['train_x'].shape[2]
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    np.random.seed(0)
    torch.manual_seed(0)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(0)
        
    batch_size = 32

    for w in tqdm(windows):
        
        x_tr = w['train_x'] 
        y_tr = w['train_y']  
        
        x_v = w['val_x'] 
        y_v = w['val_y'] 
        
        x_c = w['comb_x'] 
        y_c = w['comb_y']  
        
        x_te = w['test_x'] 
        y_te = w['test_y']  

        train_dataset = TensorDataset(torch.tensor(x_tr, dtype=torch.float32), 
                                      torch.tensor(y_tr, dtype=torch.float32))
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
        
        val_dataset = TensorDataset(torch.tensor(x_v, dtype=torch.float32), 
                                    torch.tensor(y_v, dtype=torch.float32))
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
                
        comb_dataset = TensorDataset(torch.tensor(x_c, dtype=torch.float32), 
                                    torch.tensor(y_c, dtype=torch.float32))
        comb_loader = DataLoader(comb_dataset, batch_size=batch_size, shuffle=False)
        
        test_dataset = TensorDataset(torch.tensor(x_te, dtype=torch.float32), 
                                     torch.tensor(y_te, dtype=torch.float32))
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)        

        best_pars = []
        best_e = np.inf
        for lr in [1e-3]:
            if rnn == 'jnn':
                model = jordan_rnn(input_size=input_size, hidden_size=hidden_size).to(device)
            elif rnn == 'enn':
                model = elman_rnn(input_size=input_size, hidden_size=hidden_size).to(device)
            elif rnn == 'lstm':
                model = lstm_rnn(input_size=input_size, hidden_size=hidden_size).to(device)
            elif rnn == 'gru':
                model = gru_rnn(input_size=input_size, hidden_size=hidden_size).to(device)
                
            loss_fn = nn.MSELoss(reduction='mean')
            optimizer = optim.Adam(model.parameters(), lr=lr)  
            
            for epoch in range(200):
                train(model, device, train_loader, loss_fn, optimizer) 
                if (epoch + 1) % 50 == 0:
                    e_v = test(model, device, val_loader, loss_fn) 
                    if best_e > e_v:
                        best_e = e_v
                        best_pars = [epoch+1, lr]
        
        params.append(best_pars)
        if rnn == 'jnn':
            model = jordan_rnn(input_size=input_size, hidden_size=hidden_size).to(device)
        elif rnn == 'enn':
            model = elman_rnn(input_size=input_size, hidden_size=hidden_size).to(device)
        elif rnn == 'lstm':
            model = lstm_rnn(input_size=input_size, hidden_size=hidden_size).to(device)
        elif rnn == 'gru':
            model = gru_rnn(input_size=input_size, hidden_size=hidden_size).to(device)
            
        loss_fn = nn.MSELoss(reduction='mean')
        optimizer = optim.Adam(model.parameters(), lr=best_pars[1])  
        
        for epoch in range(best_pars[0]):
            train(model, device, comb_loader, loss_fn, optimizer) 
        err, pred, targ = test(model, device, test_loader, loss_fn, predict=True, y_scaler = w['test_y_scaler'])
        
        #errors.append(err)
        preds.append(pred)
        targets.append(targ)
        
        window_metrics = add_metrics(targ, pred, window_metrics, metrics)

    preds = np.concatenate(preds)     
    targets = np.concatenate(targets) 
    result = calculate_metrics(targets, preds, metrics)   
    
    return result, window_metrics, params, preds, targets #errors



In [67]:
# in this cell, rnn, n_hidden, len_seq, and to_scale were altered (with corresponding changes for description and result path)

for t in ticker_list:      
    for features, set_name in zip(features_names, ['rv', 'all']):
        result, window_metrics, params, preds, targets = run_rnn('jnn', get_windows_rnn(data[t], features, to_scale=True, len_seq=3), hidden_size=20, metrics = ['mse', 'rmse', 'mae', 'mape', 'r2'])
        description = {'model': 'Jordan',
                       'ticker': t,
                       'data_scaled': True,
                       'data_set': set_name,
                       'rolling_window': '24m3m3m',
                       'sequence_length': 3, 
                       'n_hidden': 20,
                       'max_epoch': 200,
                       'result': result,
                       'window_metrics': window_metrics,
                       'parameters': params,
                       'predictions': preds.tolist(),
                       'targets': targets.tolist()}
    
        res_path = 'results/J_s_3_20/' + t + '_' + set_name + '.json'
    
        with open(res_path, 'w') as f:
            json.dump(description, f)
                
    print(t)
    
print('finished')

100%|███████████████████████████████████████████| 33/33 [02:45<00:00,  5.02s/it]


SBER


100%|███████████████████████████████████████████| 33/33 [02:48<00:00,  5.12s/it]


GAZP


100%|███████████████████████████████████████████| 33/33 [02:59<00:00,  5.44s/it]


...

GMKN


100%|███████████████████████████████████████████| 33/33 [02:56<00:00,  5.34s/it]


TATN


100%|███████████████████████████████████████████| 33/33 [03:06<00:00,  5.65s/it]

SNGS
finished
